In [1]:
### This will almost certainly need customization depending on the imput format of the CSV/data file
## But basically, this will load cell masks and store them as DSA annotations
import girder_client
import json
import pandas as pd

gc=girder_client.GirderClient(apiUrl="https://imaging.htan.dev/girder/api/v1")
_ = gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [2]:
# Set information about the input datafiles and any other conversion factors
segFile = "./cellSegmentation_FOVs_0-4.csv"
coordsToPixels = 0.109  ## The coordinates are apparently in microns per pixel...
# Use pandas to load the segmentation file into a dataframe
segDF = pd.read_csv(segFile)
#segDF.set_index('cellID',inplace=True) 
boundaries = segDF.to_dict(orient='records') # While I can do all of this in pandas, I like just making everything a dict

In [3]:
imageId = '6036bfd31e6134859db73031'

In [4]:
annotationDocList = gc.get(
        "/annotation?itemId=" + imageId)

for adl in annotationDocList:
    print(adl['annotation']['name'])

demo_mostly_lymphocytic_infiltrate-0
demo_mostly_lymphocytic_infiltrate-0
demo_mostly_lymphocytic_infiltrate-0
ShapeTest
zPlaneSeg


In [5]:
gc.get("/annotation/%s" % adl['_id'])

{'_accessLevel': 2,
 '_id': '603afccec433a468c7a55a2d',
 '_modelType': 'annotation',
 '_version': 149,
 'annotation': {'description': 'TestLoad',
  'elements': [{'fillColor': 'rgba(0,0,0,0)',
    'lineColor': 'rgb(63,75,120)',
    'lineWidth': 1,
    'type': 'polyline',
    'closed': True,
    'points': [[9662, 25264, 0],
     [9663, 25264, 0],
     [9664, 25264, 0],
     [9665, 25265, 0],
     [9665, 25265, 0],
     [9666, 25265, 0],
     [9667, 25266, 0],
     [9667, 25266, 0],
     [9668, 25266, 0],
     [9668, 25266, 0],
     [9669, 25265, 0],
     [9670, 25265, 0],
     [9671, 25265, 0],
     [9671, 25265, 0],
     [9672, 25264, 0],
     [9673, 25264, 0],
     [9673, 25264, 0],
     [9674, 25263, 0],
     [9674, 25263, 0],
     [9675, 25262, 0],
     [9675, 25262, 0],
     [9676, 25261, 0],
     [9677, 25261, 0],
     [9677, 25261, 0],
     [9678, 25260, 0],
     [9678, 25260, 0],
     [9679, 25259, 0],
     [9679, 25259, 0],
     [9680, 25258, 0],
     [9680, 25258, 0],
     [968

In [ ]:
annotationDocList = gc.post(
        "/annotation?itemId=" + imageId, json=annotationDoc)


In [ ]:
### Make it a polyline.. each element is below..

In [8]:
# for idx,plane in enumerate(currentCell['x_boundary']):
#     if plane:
#         print(idx, [int(x/coordsToPixels) for x in plane[0]])

zPlane = 3

elementList = []
fillColor = 'rgba(0,0,0,0)'
lineColor = 'rgb(63,75,120)'

spxDataArray = []


for idx, cellBounds in enumerate(boundaries):
    elementData = {'fillColor': fillColor,
    'lineColor': lineColor,
    'lineWidth': 1,
    'type': 'polyline',
    'closed': True, 'points':[], 'label':{'value':cellBounds['cellID']}}
    pointList = []
    spxPointList = ""

    try:
        for x,y in zip(json.loads(cellBounds['x_boundary'])[zPlane][0],json.loads(cellBounds['y_boundary'])[zPlane][0]):
            pointList.append((int(x/coordsToPixels),int(y/coordsToPixels),0))
            spxPointList += "%d,%d " % (int(x/coordsToPixels),int(y/coordsToPixels))
        
    except:
        pass
        #print("No bounds found for zPlane %d " % zPlane)
    if pointList:
    
        elementData['points'] = pointList
        elementList.append(elementData)
        spxDataArray.append({"y_cent":-1,"x_cent":-1,"spxBoundaries": spxPointList, "label":idx})

In [10]:
with open("merFishCellMask.json","w") as fp:
    json.dump(spxDataArray,fp)

In [ ]:

annotationDoc = {'name': 'zPlaneSeg',
 'description': 'TestLoad',
 'elements': elementList}
annotationDocList = gc.post(
        "/annotation?itemId=" + imageId, json=annotationDoc)